In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV,StratifiedKFold,cross_validate
from sklearn.decomposition import PCA
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.pipeline import Pipeline
from scipy.stats import loguniform,uniform,randint
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
#准备文件夹保存数据
local_path=os.getcwd()
if not os.path.exists(local_path+'\\blokagedata'):
    os.mkdir(local_path+'\\blokagedata')
print(local_path+'\\blokagedata')
#读取模型数据
sample=pd.read_csv(local_path+'\\blokagedata\\Sample_116.csv',header=0)
#划分样本和标签
x=np.array(sample.iloc[:,range(0,96)])#.astype(int)
y=np.array(sample.loc[:,'code']).reshape(-1,1).astype(int)

In [ ]:
pipe = Pipeline([('scaler', QuantileTransformer(output_distribution = 'uniform')), ('rfc', RandomForestClassifier(criterion='gini',))])
param_dist = {'rfc__n_estimators': randint(10, 1000),
 'rfc__min_samples_split': randint(2, 1000),
 'rfc__max_depth': randint(2, 1000),
 'rfc__min_samples_leaf': randint(2, 1000)}
splitter=StratifiedKFold(n_splits=5,random_state=1,shuffle=True)
clf = RandomizedSearchCV(estimator=pipe,param_distributions=param_dist,n_iter=1000,cv=splitter,random_state=1,n_jobs=-1,scoring='accuracy',verbose=3)
search = clf.fit(x, y.ravel())

In [ ]:
print('网格搜索-度量记录：',search.cv_results_)  # 包含每次训练的相关信息
print('网格搜索-最佳度量值:',search.best_score_)  # 获取最佳度量值
print('网格搜索-最佳参数：',search.best_params_)  # 获取最佳度量值时的代定参数的值。是一个字典
print('网格搜索-最佳模型：',search.best_estimator_)  # 获取最佳度量时的分类器模型

In [ ]:
q1=search.cv_results_['param_rfc__n_estimators'].data.astype(int)
q2=search.cv_results_['param_rfc__min_samples_split'].data.astype(int)
q3=search.cv_results_['param_rfc__max_depth'].data.astype(int)
q4=search.cv_results_['param_rfc__min_samples_leaf'].data.astype(int)
zz=search.cv_results_['mean_test_score'].astype(float)
q1=np.expand_dims(q1,axis=1)
q2=np.expand_dims(q2,axis=1)
q3=np.expand_dims(q3,axis=1)

zz=np.expand_dims(zz,axis=1)
print(q1.shape)
print(q2.shape)
print(q3.shape)
print(zz.shape)
pd.DataFrame(np.hstack((q1,q2,q3,zz))).to_csv(local_path+'\\blokagedata\\rfc_finetune.csv',encoding='ascii',index=False)

In [ ]:
xx=search.cv_results_['param_svc__C'].data.astype(float)
yy=search.cv_results_['param_svc__gamma'].data.astype(float)
zz=search.cv_results_['mean_test_score'].astype(float)
xx=np.expand_dims(xx,axis=1)
yy=np.expand_dims(yy,axis=1)
zz=np.expand_dims(zz,axis=1)
print(xx.shape)
print(yy.shape)
print(zz.shape)
zxc=pd.DataFrame(np.hstack((xx,yy,zz)))
zxc.columns=['C','gamma','acc']
zxc=pd.pivot_table(zxc,index=['C'],columns=['gamma'],values=['acc'])
zxc.to_csv(local_path+'\\blokagedata\\svm_finetune_grid_b.csv',encoding='ascii',index=True)